In [25]:
import pandas as pd
import os
import datetime
import numpy as np
from typing import List

SECOND = 1000

In [4]:
#this is an old function we are not using this anymore
def getMeanXYZ(df):
    if df.empty:
        return 
    #removing excess data so that data falls in 1 second data intervals
    maxIndx = int(len(df)/100)*100
    df = df.iloc[0:maxIndx]
    meanPerSecondDf = pd.DataFrame()
    
    for i in range(0,int(maxIndx/100)):
        startIndx = i*100
        endIndx = startIndx+100
        tempdf = df.iloc[startIndx:endIndx]
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
        
    return meanPerSecondDf


In [5]:
def getMeanXYZ_updated(df):
    startInd = 0
    endInd = 1

    meanPerSecondDf = pd.DataFrame()
    updateFlag = False
    
    #running this for loop for the length of the dataframe
    for i in range(0, len(df)):
        """As we are getting in seconds rather than milliseconds
        We confrimed this using https://www.epochconverter.com"""
        #Setting updateFlag to indicated window for data
        if(df.iloc[i].time - df.iloc[startInd].time > 1000):
            updateFlag = True

        #taking the window and calculating std and mean based in window
        if updateFlag==True:
            updateFlag = False
            tempdf = df.iloc[startInd:i-1]
            startInd = i

            meanDf = tempdf.mean(axis=0)
            stdDf = tempdf.std()
            meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
    return meanPerSecondDf

In [6]:
#trying out stuff

#need to do this for all files
#reading the file with no header
df = pd.read_csv("Data/G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv", header=None)
#renaming the columns
df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
meanDf = getMeanXYZ(df)



In [7]:
"""
In this cell trying to figure out how to write the function: getMeanXYZ_updated
"""

# temp = pd.to_datetime(df['time'].astype('str'), format ="%d/%m %H:%M:%S")
temp = df.time
tempDf = pd.DataFrame()

startInd = 0
endInd = 1

meanPerSecondDf = pd.DataFrame()

updateFlag = False
for i in range(0, len(df)):
    if(df.iloc[i].time - df.iloc[startInd].time > 60):
        updateFlag = True
    
    if updateFlag==True:
        updateFlag = False
        tempdf = df.iloc[startInd:i-1]
        startInd = i
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
meanPerSecondDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z
0,-1.175955,NaN,3.323090,NaN,8.359816,NaN
1,-0.273232,0.090832,2.524550,0.056997,8.912466,0.080559
2,-0.444276,0.130098,2.394421,0.149198,8.747410,0.497712
3,-1.386716,0.499137,1.396496,1.386063,6.473738,0.529306
4,-0.326920,0.273794,-4.757706,1.973065,3.909470,0.187076
...,...,...,...,...,...,...
111,-2.874023,0.129025,1.831791,0.313955,7.971623,0.094056
112,-2.581352,0.086911,1.876737,0.220258,8.058283,0.130200
113,-2.638912,0.131390,2.037164,0.346737,8.420490,0.112269
114,-2.155518,0.932770,1.760140,0.965102,8.746013,0.809792


In [8]:
#main code
dataPath = 'Data/'
handWashSubString = 'left-hand_wash'
notHandWashSubstring = 'left-not_hand_wash'

handWashedDf = pd.DataFrame()
notHandWashedDF = pd.DataFrame()

#Reading through the data folder for filenames
for path in os.listdir(dataPath):
    filepath = dataPath+path
    df = pd.read_csv(filepath, header=None)
    df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
    
    meanDf = getMeanXYZ_updated(df)   #using the updated getMeanXYZ function
        
    if handWashSubString in path:
        handWashedDf = handWashedDf.append(meanDf,ignore_index=True)
    else:
        notHandWashedDF = notHandWashedDF.append(meanDf,ignore_index=True)

In [9]:
handWashedDf['Activity'] = 'hand_wash'
handWashedDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,1.652931,3.442638,-5.572326,6.064842,-0.031856,7.454768,hand_wash
1,-1.807909,3.107468,-9.217029,3.334817,-3.186268,5.243445,hand_wash
2,-1.296940,2.364967,-9.333873,3.010561,-2.229245,4.290689,hand_wash
3,-1.321506,0.646286,-9.476633,0.899180,-2.606725,0.862985,hand_wash
4,-1.377510,1.219219,-9.081902,2.117393,-1.758705,3.397296,hand_wash
...,...,...,...,...,...,...,...
275,-2.635623,2.249142,-7.191513,3.312726,-3.064116,5.048388,hand_wash
276,-2.152636,2.390652,-8.213645,2.889089,-6.445359,3.837324,hand_wash
277,-2.203953,3.969165,-7.823268,4.079654,-4.716966,8.721495,hand_wash
278,-2.386131,3.235715,-6.978507,4.759580,-1.289175,6.595727,hand_wash


In [10]:
notHandWashedDF['Activity'] = 'no_hand_wash'
notHandWashedDF

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,-7.305590,2.992347,-3.954621,3.668469,0.847393,5.102091,no_hand_wash
1,-10.937159,4.566015,-5.572823,3.025889,-2.797383,1.618498,no_hand_wash
2,-4.380031,10.722338,-7.941981,6.268560,-1.095552,6.428559,no_hand_wash
3,5.031961,11.984091,0.341984,6.438999,4.860490,5.546686,no_hand_wash
4,3.104889,9.827950,-4.112285,10.074004,0.416818,12.775790,no_hand_wash
...,...,...,...,...,...,...,...
659,5.533239,14.328041,1.929882,15.547365,-0.020017,7.301156,no_hand_wash
660,7.636181,12.836038,4.101955,12.189794,4.459284,8.174824,no_hand_wash
661,1.154762,20.572129,13.743348,17.464195,5.574227,13.963414,no_hand_wash
662,4.055239,18.315336,4.166200,16.059836,2.329742,10.986047,no_hand_wash


In [11]:
nameOfTheFile = 'processedDataAssignment1.csv'
to_save_df = pd.DataFrame()
to_save_df = to_save_df.append(handWashedDf,ignore_index=True)
to_save_df = to_save_df.append(notHandWashedDF,ignore_index=True)

#removing data points that are empty
to_save_df['std_x'].replace('', np.nan, inplace=True)
to_save_df.dropna(subset=['std_x'], inplace=True)

#saving the dataframe to a file as defined in nameOfTheFile
to_save_df.to_csv(nameOfTheFile, index=False,encoding='utf-8')

# Project part 2

In [12]:
def getMeanXYZSlidingWindow(df, timeWindow):
    meanPerSecondDf = getMeanXYZ_updated(df)
    slidingWindowDf = pd.DataFrame()
    
    for i in range(0, len(meanPerSecondDf)-timeWindow):    
        tempdf = meanPerSecondDf.iloc[i:(i+timeWindow)]
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        
#         slidingWindowDf = meanDf
        slidingWindowDf = slidingWindowDf.append({'mean_x':meanDf.mean_x, 'std_x':stdDf.std_x, 'mean_y':meanDf.mean_y,'std_y':stdDf.std_y, 'mean_z':meanDf.mean_z, 'std_z':stdDf.std_z},ignore_index=True)
    return slidingWindowDf

In [13]:
#trial
#reading the file with no header
df = pd.read_csv("Data/G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv", header=None)
df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')

slidingWindow = getMeanXYZSlidingWindow(df,2)
slidingWindow

,mean_x,std_x,mean_y,std_y,mean_z,std_z
0,-0.077489,0.237001,-7.394677,1.930419,-1.609062,1.563641
1,-1.552424,0.525027,-9.275451,0.229284,-2.707756,0.673700
2,-1.309223,1.215291,-9.405253,1.492972,-2.417985,2.423753
3,-1.349508,0.405124,-9.279267,0.861407,-2.182715,1.792028
4,-1.549555,0.308767,-9.163055,0.271962,-1.818805,0.114616
5,-1.881683,0.877721,-8.289133,1.403986,-0.492570,2.679444


In [14]:
#renaming new files
folder = r"C:\Users\gawal\smartWatch\newData" #This is an absolute path!!!
for filename in os.listdir(dataPath):
    oldName = os.path.join(folder,filename)
    
    newName = os.path.join(folder, filename.replace('.csv', '-new.csv'))
    print(newName)
    os.rename(oldName, newName)

C:\Users\gawal\smartWatch\newData\G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42-new.csv


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\gawal\\smartWatch\\newData\\G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv' -> 'C:\\Users\\gawal\\smartWatch\\newData\\G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42-new.csv'

In [15]:
def combineDataAndWriteToFile(nameOfTheFile, handWashedDf, notHandWashedDF):
     #saving the files
    to_save_df = pd.DataFrame()
    to_save_df = to_save_df.append(handWashedDf,ignore_index=True)
    to_save_df = to_save_df.append(notHandWashedDF,ignore_index=True)
    
    #removing data points that are empty
    to_save_df['std_x'].replace('', np.nan, inplace=True)
    to_save_df.dropna(subset=['std_x'], inplace=True)
    
    #saving the dataframe to a file as defined in nameOfTheFile
    to_save_df.to_csv(nameOfTheFile, index=False,encoding='utf-8')

In [16]:
def rms(x):
    rms = np.sqrt(np.mean(x**2))
    return rms

In [17]:
def getProcessedValues(df, timeWindow):
    
    startInd = 0
    newStartInd = startInd

    meanPerSecondDf = pd.DataFrame()
    calculateValues = False
    updateStartIndex = False
    
    #running this for loop for the length of the dataframe
    for i in range(0, len(df)):
        """As we are getting in seconds rather than milliseconds
        We confrimed this using https://www.epochconverter.com"""
        #Setting updateFlag to indicated window for data
        if(df.iloc[i].time - df.iloc[startInd].time > 1000*timeWindow):
            calculateValues = True
        
        if(not updateStartIndex and df.iloc[i].time - df.iloc[startInd].time > 1000):
            updateStartIndex = True
            newStartInd = i

        #taking the window and calculating std and mean based in window
        if calculateValues==True:
            calculateValues = False
            updateStartIndex = False
            tempdf = df.iloc[startInd:i-1]
            startInd = newStartInd

            meanDf = tempdf.mean(axis=0)
            stdDf = tempdf.std()
            medianDf = tempdf.median()
            meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'median_x':medianDf.X, 'rms_X':rms(tempdf.X),  
                                                      'mean_y':meanDf.Y,'std_y':stdDf.Y, 'median_y':medianDf.Y, 'rms_Y':rms(tempdf.Y),
                                                      'mean_z':meanDf.Z, 'std_z':stdDf.Z, 'median_z':medianDf.Z, 'rms_Z':rms(tempdf.Z)}, 
                                                     ignore_index=True)
    return meanPerSecondDf

In [18]:
#trying out stuff

#need to do this for all files
#reading the file with no header
df = pd.read_csv("Data/G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv", header=None)
#renaming the columns
df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
meanDf = getProcessedValues(df,3)


meanDf

,mean_x,std_x,median_x,rms_X,mean_y,std_y,median_y,rms_Y,mean_z,std_z,median_z,rms_Z
0,-0.520780,3.340954,-0.607137,3.375400,-8.075292,4.626338,-8.672964,9.302521,-1.891787,5.897741,-1.622626,6.183686
1,-1.462147,2.268242,-1.274150,2.695399,-9.334900,2.623727,-9.327403,9.695397,-2.683804,3.911404,-2.568060,4.738089
2,-1.476791,1.249618,-1.432222,1.933165,-9.261667,1.962914,-9.342971,9.466698,-2.069616,2.774421,-2.374662,3.457521
3,-1.716600,2.066858,-1.551973,2.684033,-8.524116,3.312588,-8.671168,9.143103,-0.936146,5.035505,-1.942361,5.113329


### This is for Part Two
##### Getting the values of Mean and Std and using sliding window

In [19]:
#main code
dataPath = 'Data/'
handWashSubString = 'left-hand_wash'
notHandWashSubstring = 'left-not_hand_wash'

handWashedDf = pd.DataFrame()
notHandWashedDF = pd.DataFrame()

for windowLength in range(2,5):
    
    nameOfTheFile = 'processedDataAssignment2Window' + str(windowLength) + '.csv' 
    handWashSubString = 'left-hand_wash'
    notHandWashSubstring = 'left-not_hand_wash'

    handWashedDf = pd.DataFrame()
    notHandWashedDF = pd.DataFrame()
    
    #Reading through the data folder for filenames
    for path in os.listdir(dataPath):
        filepath = dataPath+path
        df = pd.read_csv(filepath, header=None)
        df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')

        meanDf = getMeanXYZSlidingWindow(df, windowLength)   #using the updated getMeanXYZ function

        if handWashSubString in path:
            handWashedDf = handWashedDf.append(meanDf,ignore_index=True)
        else:
            notHandWashedDF = notHandWashedDF.append(meanDf,ignore_index=True)
            
    handWashedDf['Activity'] = 'hand_wash'
    notHandWashedDF['Activity'] = 'no_hand_wash'
    combineDataAndWriteToFile(nameOfTheFile, handWashedDf, notHandWashedDF)
   
    
    

### This is for Part Thre
##### Getting the values of Mean, Std, Median, RMS using sliding window

In [20]:
#main code
dataPath = 'Data/'
handWashSubString = 'left-hand_wash'
notHandWashSubstring = 'left-not_hand_wash'

handWashedDf = pd.DataFrame()
notHandWashedDF = pd.DataFrame()

for windowLength in range(2,5):
    
    nameOfTheFile = 'processedDataAssignment2Window_withMedianAndRMS' + str(windowLength) + '.csv' 
    handWashSubString = 'left-hand_wash'
    notHandWashSubstring = 'left-not_hand_wash'

    handWashedDf = pd.DataFrame()
    notHandWashedDF = pd.DataFrame()
    
    #Reading through the data folder for filenames
    for path in os.listdir(dataPath):
        filepath = dataPath+path
        df = pd.read_csv(filepath, header=None)
        df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')

        meanDf = getProcessedValues(df, windowLength)   #using the updated getMeanXYZ function

        if handWashSubString in path:
            handWashedDf = handWashedDf.append(meanDf,ignore_index=True)
        else:
            notHandWashedDF = notHandWashedDF.append(meanDf,ignore_index=True)
            
    handWashedDf['Activity'] = 'hand_wash'
    notHandWashedDF['Activity'] = 'no_hand_wash'
    combineDataAndWriteToFile(nameOfTheFile, handWashedDf, notHandWashedDF)
   
    
    

# Converting it to arff

In [35]:
def save_features_to_arff(features: pd.DataFrame, path: str):
    '''
    saves features to arff file

    :param features: dataframe of features
        ===============================
        feature_1 | feature_2 | ... | label
        ===============================
    :param path: path to save to
    :return: None
    '''
    attribute_list = features.columns
#     class_list = features['label'].unique()
    class_list = features['Activity'].unique()
    arff_header = make_arff_header(attribute_list, class_list)
    with open(path, 'w+') as f:
        f.write(arff_header)
        features.to_csv(f, index=False, header=False)

In [36]:
def make_arff_header(attribute_list: List[str], class_list: List[str]) -> str:
    '''
    makes arff header

    :param attribute_list: list of attributes
    :param class_list: list of classes
    :return: arff header
    '''
    header = "@Relation wada\n\n"
    for attribute in attribute_list[:-1]:
        header += f"@ATTRIBUTE {attribute} numeric\n"
    header += f"@ATTRBUTE label {{{','.join(class_list)}}}\n\n"
    header += "@DATA\n"
    return header

In [39]:
arff_path = f'processedData/Window_withMedianAndRMS2Sec.arff'
df_1 = pd.read_csv('processedDataAssignment2Window_withMedianAndRMS2.csv')
save_features_to_arff(df_1, arff_path)

arff_path = f'processedData/Window_withMedianAndRMS3Sec.arff'
df_1 = pd.read_csv('processedDataAssignment2Window_withMedianAndRMS3.csv')
save_features_to_arff(df_1, arff_path)

arff_path = f'processedData/Window_withMedianAndRMS4Sec.arff'
df_1 = pd.read_csv('processedDataAssignment2Window_withMedianAndRMS4.csv')
save_features_to_arff(df_1, arff_path)

# df_1.columns

In [34]:
data_top = df_1.head()
data_top

,mean_x,std_x,median_x,rms_X,mean_y,std_y,median_y,rms_Y,mean_z,std_z,median_z,rms_Z,Activity
0,-0.114483,3.696535,-0.425116,3.688199,-7.426707,5.169795,-8.499925,9.040836,-1.706654,6.593914,-1.373544,6.793731,hand_wash
1,-1.535900,2.758310,-1.207689,3.150815,-9.256339,3.170307,-8.956176,9.781529,-2.713938,4.785931,-2.499203,5.491019,hand_wash
2,-1.323090,1.717252,-1.223855,2.164383,-9.431271,2.201814,-9.406439,9.683607,-2.398375,3.103658,-2.493215,3.916120,hand_wash
3,-1.355294,0.971237,-1.422642,1.665926,-9.270681,1.639684,-9.355545,9.413839,-2.158456,2.519537,-2.359094,3.312795,hand_wash
4,-1.541369,1.453768,-1.452579,2.116202,-9.107986,2.280297,-8.875942,9.387663,-1.893344,3.265359,-1.985471,3.767237,hand_wash
